# **Twitter Step 3: Exploratory Data Analysis (EDA) and Preliminary Visualizations**

* Explore the collected and cleaned dataset
* Do some basic visualizations to understand the dataset characteristics

Reference:

* https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/

# **0. Setup Environment**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 266kB 8.6MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 604kB 13.3MB/s 
     |████████████████████████████████| 3.1MB 19.0MB/s 
     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 112kB 34.2MB/s 
     |████████████████████████████████| 296kB 40.0MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27085 sha256=3101f3731de2a51b81edaa3e2f59d0164811d7ed8e4d406b36d2ef5091bc2a8b
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
Successfully built htmlmin
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5
  Found existing installation: pandas-profiling 1.4.1
    Uninstal

## You want to restart the kernel at this point (watch for warning in output of last code cell above)

In [ ]:
from pathlib import Path

import requests

import pandas_profiling
from pandas_profiling.utils.cache import cache_file

## You will need to give permission for this Colab to link to your gdrive in the code cell below

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
# CUSTOMIZE: if you want your work and twitter datasets saved into a specific folder
#            beneath your gdrive root directory, define it below

%cd ./MyDrive/courses/2020f_iphs200_programming_humanity/code/

/gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code


In [ ]:
!ls *.csv

cleaned_tweets_combined_20201201-012404.csv
tweets_combined_20201201-012404.csv
tweets_seattle_all.csv
tweets_twint_donald_trump__20201201-012233.csv
tweets_twint_election_win__20201201-012219.csv
tweets_twint_future__20201201-012121.csv
tweets_twint_vote_court__20201201-012309.csv


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [ ]:
import os
import re
import glob


# **1. Read in Combined Tweet Dataset File**

In [ ]:
!ls *.csv

cleaned_tweets_combined_20201201-012404.csv
tweets_combined_20201201-012404.csv
tweets_seattle_all.csv
tweets_twint_donald_trump__20201201-012233.csv
tweets_twint_election_win__20201201-012219.csv
tweets_twint_future__20201201-012121.csv
tweets_twint_vote_court__20201201-012309.csv


In [ ]:
# CONFIGURE: Set the 'file_name_all' to the name of the combined datafile with all the tweets
#            which should be listed in the previous code cell

file_name_all = 'cleaned_tweets_combined_20201201-012404.csv'

In [ ]:
combined_df = pd.read_csv(file_name_all, encoding='utf-8')
combined_df = combined_df.convert_dtypes()
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1060 entries, 0 to 1059
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1060 non-null   Int64  
 1   id               1060 non-null   Int64  
 2   conversation_id  1060 non-null   Int64  
 3   created_at       1060 non-null   string 
 4   date             1060 non-null   string 
 5   time             1060 non-null   string 
 6   timezone         1060 non-null   Int64  
 7   user_id          1060 non-null   Int64  
 8   username         1060 non-null   string 
 9   name             1060 non-null   string 
 10  place            0 non-null      Int64  
 11  tweet            1060 non-null   string 
 12  language         1060 non-null   string 
 13  mentions         1060 non-null   string 
 14  urls             1060 non-null   string 
 15  photos           1060 non-null   string 
 16  replies_count    1060 non-null   Int64  
 17  retweets_count

In [ ]:
combined_df.shape

(1060, 41)

In [ ]:
combined_df.head()

,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,emojis,smileys,numbers,tweet_clean
0,0,1332356759994970113,1332356759994970113,2020-11-27 16:13:00+00:00,2020-11-27,16:13:00,0,939091,joebiden,Joe Biden,<NA>,"This Native American Heritage Day, we give thanks to our Indigenous communities and their ancestors. As we celebrate their rich heritage and contributions, let’s commit to writing a new future together — one built on a strong partnership and filled with opportunity for all.",en,[],[],[],6351,21601,248367,[],[],https://twitter.com/JoeBiden/status/1332356759994970113,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],this native american heritage day we give thanks to our indigenous communities and their ancestors as we celebrate their rich heritage and contributions let us commit to writing a new future together one built on a strong partnership and filled with opportunity for all
1,1,1323473727447654400,1323473727447654400,2020-11-03 03:55:00+00:00,2020-11-03,03:55:00,0,939091,joebiden,Joe Biden,<NA>,I’ve said it many times: I’m more optimistic about America’s future today than I was when I got elected to the United States Senate as a 29-year-old.,en,[],[],[],4359,5375,79312,[],[],https://twitter.com/JoeBiden/status/1323473727447654400,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[' 29'],i have said it many times i am more optimistic about americas future today than i was when i got elected to the united states senate as a year old
2,2,1323391493885718528,1323391493885718528,2020-11-02 22:28:14+00:00,2020-11-02,22:28:14,0,939091,joebiden,Joe Biden,<NA>,I’m speaking with members of the African American community in Pittsburgh about the power of the vote — and the future we can build together. Tune in. https://t.co/1wFBiLoCWu,en,[],['https://t.co/1wFBiLoCWu'],[],1818,1994,13823,[],[],https://twitter.com/JoeBiden/status/1323391493885718528,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],i am speaking with members of the african american community in pittsburgh about the power of the vote and the future we can build together tune in
3,3,1322959086544211973,1322959086544211973,2020-11-01 17:50:00+00:00,2020-11-01,17:50:00,0,939091,joebiden,Joe Biden,<NA>,We can build a future where: - Health care is a right - We end the gun violence epidemic - We combat climate change - Our government works for everyone Vote. https://t.co/eoxT07d7QB,en,[],['https://t.co/eoxT07d7QB'],[],1603,2581,18087,[],[],https://twitter.com/JoeBiden/status/1322959086544211973,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],we can build a future where health care is a right we end the gun violence epidemic we combat climate change our government works for everyone vote
4,4,1322927509260902401,1322927509260902401,2020-11-01 15:44:31+00:00,2020-11-01,15:44:31,0,939091,joebiden,Joe Biden,<NA>,The future of our planet is on the ballot. Vote: https://t.co/eoxT07d7QB,en,[],['https://t.co/eoxT07d7QB'],[],2155,4513,43222,[],[],https://twitter.com/JoeBiden/status/1322927509260902401,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],the future of our planet is on the ballot vote


# **2. Pandas-Profiling Generated Report**
By: Jon Chun
30 Nov 2020

Lo/No-code solution for Pandas EDA and simple visualizations.
References:

* https://github.com/pandas-profiling/pandas-profiling
* https://colab.research.google.com/github/pandas-profiling/pandas-profiling/blob/master/examples/meteorites/meteorites.ipynb#scrollTo=KbIMcOpp_rfh


In [ ]:
# Option #1 to display Pandas-Profiling Report

report = combined_df.profile_report(
    sort="None", html={"style": {"full_width": True}}, progress_bar=False
)
report

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
combined_df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,emojis,smileys,numbers,tweet_clean
0,1332356759994970113,1332356759994970113,2020-11-27 16:13:00+00:00,2020-11-27,16:13:00,0,939091,joebiden,Joe Biden,<NA>,"This Native American Heritage Day, we give thanks to our Indigenous communities and their ancestors. As we celebrate their rich heritage and contributions, let’s commit to writing a new future together — one built on a strong partnership and filled with opportunity for all.",en,[],[],[],6351,21601,248367,[],[],https://twitter.com/JoeBiden/status/1332356759994970113,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],this native american heritage day we give thanks to our indigenous communities and their ancestors as we celebrate their rich heritage and contributions let us commit to writing a new future together one built on a strong partnership and filled with opportunity for all
1,1323473727447654400,1323473727447654400,2020-11-03 03:55:00+00:00,2020-11-03,03:55:00,0,939091,joebiden,Joe Biden,<NA>,I’ve said it many times: I’m more optimistic about America’s future today than I was when I got elected to the United States Senate as a 29-year-old.,en,[],[],[],4359,5375,79312,[],[],https://twitter.com/JoeBiden/status/1323473727447654400,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[ 29],i have said it many times i am more optimistic about americas future today than i was when i got elected to the united states senate as a year old
2,1323391493885718528,1323391493885718528,2020-11-02 22:28:14+00:00,2020-11-02,22:28:14,0,939091,joebiden,Joe Biden,<NA>,I’m speaking with members of the African American community in Pittsburgh about the power of the vote — and the future we can build together. Tune in. https://t.co/1wFBiLoCWu,en,[],[https://t.co/1wFBiLoCWu],[],1818,1994,13823,[],[],https://twitter.com/JoeBiden/status/1323391493885718528,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],i am speaking with members of the african american community in pittsburgh about the power of the vote and the future we can build together tune in
3,1322959086544211973,1322959086544211973,2020-11-01 17:50:00+00:00,2020-11-01,17:50:00,0,939091,joebiden,Joe Biden,<NA>,We can build a future where: - Health care is a right - We end the gun violence epidemic - We combat climate change - Our government works for everyone Vote. https://t.co/eoxT07d7QB,en,[],[https://t.co/eoxT07d7QB],[],1603,2581,18087,[],[],https://twitter.com/JoeBiden/status/1322959086544211973,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],we can build a future where health care is a right we end the gun violence epidemic we combat climate change our government works for everyone vote
4,1322927509260902401,1322927509260902401,2020-11-01 15:44:31+00:00,2020-11-01,15:44:31,0,939091,joebiden,Joe Biden,<NA>,The future of our planet is on the ballot. Vote: https://t.co/eoxT07d7QB,en,[],[https://t.co/eoxT07d7QB],[],2155,4513,43222,[],[],https://twitter.com/JoeBiden/status/1322927509260902401,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>,[],[],[],the future of our planet is on the ballot vote


# **3. NLP Analysis**

Options

* Sentiment Analysis (FastText and/or VADER)
* Named Entity Recognition (NER using flair)

# **3. Write Reports to Output files**

In [ ]:
# Create unique output filename using current datetime stamp

file_name_edaviz = ('edaviz_' + file_name_all.split('.')[0] + '.html') # 'tweets_combined_20201201-012404.csv'
print(file_name_edaviz)

edaviz_cleaned_tweets_combined_20201201-012404.html


In [ ]:
profile_report = combined_df.profile_report(html={"style": {"full_width": True}})
profile_report.to_file(file_name_edaviz)

/usr/local/lib/python3.6/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 9918 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.6/dist-packages/matplotlib/textpath.py:84: RuntimeWarning: Glyph 9989 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 9918 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 9989 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 9918 missing from current font.
  font.set_text(s, 0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 9989 missing from current font.
  font.set_text(s, 0, fla

In [ ]:
!ls eda_viz*

eda_viz_cleaned_tweets_combined_20201201-012404.html


In [ ]:
# Option #2 to display Pandas-Profiling Report

# import IPython
# IPython.display.HTML(filename=file_name_edaviz)

In [ ]:
# Option #3 to display Pandas-Profiling Report

"""
profile_report = combined_df.profile_report(
    explorative=True, html={"style": {"full_width": True}}
)
profile_report
"""